Goal: Create a daily NAO index using the CPC methodology 
=====

Author: [Ray Bell](https://github.com/raybellwaves)

In [ ]:
# Load python packages
%matplotlib inline
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
hv.notebook_extension()

Use the time period of the [SubX](http://iridl.ldeo.columbia.edu/SOURCES/.Models/.SubX/) hindcast: 1999-2016.
For a saninty check make sure that 1999-2016 is indeed the hindcast period:

In [ ]:
remote_data = xr.open_dataset('http://iridl.ldeo.columbia.edu/SOURCES/.Models'\
                              '/.SubX/.RSMAS/.CCSM4/.hindcast/.zg/dods')
print(remote_data.coords['S'])

The data used here is geopotential height at 500 hPa ($Z_{500}$) from ERA-Interim. This data was simply copied from the University of Reading archive. You can however use the [API](https://software.ecmwf.int/wiki/display/CKB/How+to+download+data+via+the+ECMWF+WebAPI) to download the data from ECMWF.

Two files exists in this directory `ERAI_z500_monthly_1999-2016.nc` (43 Mb) and `b.nc`. `ERAI_z500_monthly_1999-2016.nc` is the monthly mean of $Z_{500}$ January 1996 - December 2016. `b.nc` is daily $Z_{500}$ January 1st 1996 - December 31st 2016. The data has been interpolated to 1° to match the SubX data (360x181) and cut to 20°N-90°N for the analysis. The reference scripts for the data processing beforehand are in the directory `pre_proc`. DO THIS!

CPC's methodology for the daily NAO index can be found [here](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/history/method.shtml), with some more information [here](http://www.cpc.ncep.noaa.gov/data/teledoc/telepatcalc.shtml). The procedure is based on [Barston and Livezey (1987)](https://journals.ametsoc.org/doi/pdf/10.1175/1520-0493%281987%29115%3C1083%3ACSAPOL%3E2.0.CO%3B2).

My understanding of the methodology is as follows:

1. Use the region of 20°N-90°N with standardized $Z_{500}$ anomalies.

2. Calculate the 10 leading EOFs for each calendar month centred on that month with a window of 3 months. e.g. February is based on January-February-March (JFM). Decisions have to be made for January and December: whether to just use the two-month window or leave out the season. (It shouldn't really matter).

3. Rotate the EOFs using the varimax methodology.

4. Linearly interpolate the monthly spatial pattern to the day in question. e.g. February 1$^{st}$ will be linearly interpolated from the DJF pattern (think of the pattern defined mid-way through the mid-month: e.g. January 15$^{th}$) and the JFM pattern (February 15$^{th}$).

5. Least squared regression approach for daily data? (e.g. last paragraph in the NAO/PNA section [here](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/history/method.shtml) (I don't understand this)

Try creating a daily index for one day e.g. February 15$^{th}$ 2009

First we need to obtain the NAO pattern from the monthly data.

Calcualte the 10 leading EOFs for all JFMs

Use xarray's [rolling mean](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.rolling.html) to do the seasonal averages. Then slice it to get all the JFM's.

In [ ]:
da = xr.open_dataarray('ERAI_z500_monthly_1999-2016.nc')
print(da)

sm = da.rolling(time=3).mean().dropna('time')
# Make note that time is now given as the last month in the window e.g. JFM has time 03-16T09 (MM-DDTHH)

# Use numpy's slice index to get all the JFM's
jfm = sm[0::12,:,:]
# Check that all the JFM 1999-2016 were correctly sliced 
print(jfm.coords['time'])

Calculate standardized anomalies

In [ ]:
jfm_sa = (jfm - jfm.mean(dim=('time'))) / (jfm - jfm.mean(dim=('time'))).std(dim=('time'))
print(jfm_sa)
print(jfm_sa.max())
print(jfm_sa.min())

Before doing the EOF apply a weighting as cosine of the latitude

In [ ]:
data = jfm_sa.values
wgts = np.sqrt(np.cos(np.deg2rad(jfm_sa.coords['lat'].values)).clip(0., 1.))[..., np.newaxis]
weights = np.broadcast_arrays(data[0:1], wgts)[1][0]
data_weighted = data * weights

Calculate the 10 leading EOF modes. For a basic understanding of EOFs I recommending reading Hannachi's EOF primer [here](http://www.met.rdg.ac.uk/~han/Monitor/eofprimer.pdf) and his paper [here](https://rmets.onlinelibrary.wiley.com/doi/full/10.1002/joc.1499). There is also a [note](http://www.met.rdg.ac.uk/~sws97mha/Eofs/eof.pdf) written by Ambaum. This code is also adapted from Dawson's [eof package](https://github.com/ajdawson/eofs)

In [ ]:
# Reshape the data to be (time, space)
records = len(jfm_sa.coords['time'])
originalshape = data_weighted.shape[1:]
channels = np.prod(originalshape)
data_weighted_flat = data_weighted.reshape([records, channels])
print(np.shape(data_weighted_flat))

In [ ]:
# Compute the singular value decomposition
# Principal component, eigenvalue, eof
A, Lh, E = np.linalg.svd(data_weighted_flat, full_matrices=False)

In [ ]:
# Construct the eigenvalues and normalize by N-1
L = (Lh * Lh) / (float(records - 1))

# Remove the scaling on the principal component time-series that is
# implicitily introduced by using SVD instead of eigen-decomposition.
# The PCs may be re-scaled later if required.
P = A * Lh 

Return the 10 dominant EOFs

In [ ]:
# Calculate the Principal Components for a subset and all
npcs = 10
sub_pcs = P[:, 0:npcs] / np.sqrt(L[0:npcs])
pcs = P / np.sqrt(L)
print(np.shape(sub_pcs))
print(np.shape(pcs))
# Put it in a DataArray
sub_pcs_da = xr.DataArray(sub_pcs, coords=[jfm_sa.coords['time'], range(sub_pcs.shape[1])],
                      dims=['time', 'mode'], name='pcs')

# Calculate the eofs for a subset and all
sub_flat_eofs = E[0:npcs, :].copy()
sub_eofs = sub_flat_eofs / np.sqrt(L[0:npcs])[:, np.newaxis]
sub_eofs2d = sub_eofs.reshape((npcs,) + originalshape)
eofs = E / np.sqrt(L)[:, np.newaxis]
print(np.shape(sub_eofs))
#print(np.shape(eofs))
print(np.shape(sub_eofs2d))

Test plotting just the 1st eof

In [ ]:
tmp = sub_eofs2d[0,:,:] * 100 # I can't explain the 100 scaling other than it matches
# the calculation of eof by NCL. See eof_5.ncl here https://www.ncl.ucar.edu/Applications/eof.shtml
tmp_da = xr.DataArray(tmp, coords=[jfm_sa.coords['lat'], jfm_sa.coords['lon']],
                         dims=['lat', 'lon'], name='z500')
ax = plt.axes(projection=ccrs.Orthographic(0, 90))
ax.coastlines()
ax.set_global()
tmp_da.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False)
plt.show()

Calculate the EOFs as covariance

In [ ]:
sub_out_shape = (npcs,) + originalshape # (10, 71, 361)
data_flat = data.reshape([records, channels]) # (18, 25560)

# Divisor
div = np.float64(sub_pcs.shape[0] - float(records - 1))
sub_cov = (np.dot(data_flat.T, sub_pcs).T / div).reshape(sub_out_shape)
# Put into xr.DataArray
sub_cov_da = xr.DataArray(sub_cov, coords=[range(sub_pcs.shape[1]), jfm_sa.coords['lat'], jfm_sa.coords['lon']],
                         dims=['mode', 'lat', 'lon'], name='z500')
print(sub_cov_da)

Plot it

In [ ]:
sub_cov_da.isel(mode=0).plot()

Make the plot nicer using [cartopy](https://github.com/SciTools/cartopy)

In [ ]:
ax = plt.axes(projection=ccrs.Orthographic(0, 90))
ax.coastlines()
ax.set_global()
sub_cov_da.isel(mode=0).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False)
plt.show()

Show the variance explained by each mode

In [ ]:
sub_varexpl = (L[0:npcs] / L.sum()) * 100
print(sub_varexpl)
print(np.sum(sub_varexpl))

Have a look at the other modes of variability using [geoviews](https://github.com/ioam/geoviews). Just move the slider.

In [ ]:
%%opts Image [projection=ccrs.Orthographic(0, 90) colorbar=False fig_size=200] (cmap='RdBu_r') Overlay [xaxis=None yaxis=None]
dataset = gv.Dataset(sub_cov_da, kdims=['mode', 'lon', 'lat'])
dataset.to(gv.Image, ['lon', 'lat']) * gf.coastline()

There are some known patterns here and others may be spurious
- 1st Looks a bit like the [Artic Oscillation](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/loading.html)
- 2nd Looks like the [Pacific North American](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/pna_loading.html)
- 3rd. Is possibly the [NAO](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/nao_loading.html) but there is a lot of activity outside of the North Atlantic. Perhaps performing the REOF will reduce the noise for this variable outside of the North Atlantic?

For refence I also computed the eofs in NCL to see if they match. The code and plot is in the `NCL` directory and are shown here.
![title](NCL/eof.png)

The idea of rotating the eofs is to maximise the sum of the variances so the coefficients will either be large or near-zero. You can read about it more in [Richman 1986](https://rmets.onlinelibrary.wiley.com/doi/pdf/10.1002/joc.3370060305) and [Mestas-Nunez, 200](http://www.aoml.noaa.gov/phod/docs/mestas-00.pdf). There some other notes about reofs [here](http://www.jsg.utexas.edu/fu/files/GEO391-W11-REOF.pdf) and some more with math examples [here](https://atmos.washington.edu/~dennis/552_Notes_4.pdf). The most common method is the varimax rotation. There is an answer for how to do with numpy [here](https://stackoverflow.com/questions/17628589/perform-varimax-rotation-in-python-using-numpy), *bmcmenamin* has a fa_kit where is does the rotation [here](https://github.com/bmcmenamin/fa_kit/blob/master/fa_kit/rotation.py) based on a script by *rossfadely* [here](https://github.com/rossfadely/consomme/blob/master/consomme/rotate_factor.py). Dawson also has the rotation as WIP [here](https://github.com/ajdawson/eofs/blob/experimental-rotation/lib/eofs/experimental/rotation/kernels.py). I hope to build on that package. See example 5 using ncl [here](https://www.ncl.ucar.edu/Applications/eof.shtml). The underlying NCL code is [here](https://github.com/yyr/ncl/blob/34bafa4a78ba69ce8852212f59546bb433ce40c6/ni/src/lib/nfpfort/varimax_dp.f) or [here](https://github.com/yyr/ncl/blob/34bafa4a78ba69ce8852212f59546bb433ce40c6/ni/src/lib/nfpfort/varimax_JiangLing_dp.f) with some chat about them [here](https://www.ncl.ucar.edu/Support/talk_archives/2012/3295.html)

# This needs work...

In [ ]:
eps = 1e-10 # Tolerance value used to determine convergence of the rotation algorithm
itermax = 1000 # Maximum number of iterations 

# Apply kaiser row normalization to sub_eofs
scale = np.sqrt((sub_eofs ** 2).sum(axis=0))
sub_eofs_norm = sub_eofs / scale

rotation = np.eye(npcs, dtype=sub_eofs_norm.dtype) # Initialize
delta = 0.
for i in range(itermax):
    z = np.dot(sub_eofs_norm.T, rotation)
    b = np.dot(sub_eofs_norm,
               z ** 3 - np.dot(z, np.diag((z ** 2).sum(axis=0)) / channels))
    u, s, vh = np.linalg.svd(b)
    rotation = np.dot(u, vh)
    delta_previous = delta
    delta = s.sum()
    if delta < delta_previous * (1. + eps): break
print(np.shape(rotation))
print(np.shape(sub_eofs_norm))
reofs = np.dot(sub_eofs_norm.T, rotation).T
# de-normalize
reofs = reofs * scale
print(np.shape(reofs))
#print(np.max(sub_eofs))
#print(np.min(sub_eofs))
#print(np.max(reofs))
#print(np.min(reofs))

In [ ]:
# See https://github.com/yyr/ncl/blob/34bafa4a78ba69ce8852212f59546bb433ce40c6/ni/src/lib/nfpfort/varimax_JiangLing_dp.f

# Put the reofs in descending order based on their variance
reofs_var = (reofs ** 2).sum(axis=1)
print(np.sum(reofs_var))
print(reofs_var)
reofs_var_des = np.flip(np.sort(reofs_var), axis=0) # There may be a better way to do this
print(reofs_var_des)
# Get the index of the sort
ix = np.flip(np.argsort(reofs_var), axis=0)
print(ix)

# Index reofs in terms of descending reofs
reofs_des = reofs[ix,:]

# Reshape to 2d
reofs_des2d = reofs_des.reshape((npcs,) + originalshape)
print(np.shape(reofs_des2d))

# Need to work out how to obtain the new rpcs. Stuck.

#reofs2d = reofs.reshape((npcs,) + originalshape)
#print(np.shape(reofs2d))
#print(np.max(reofs2d))
#print(np.min(reofs2d))

#(L[0:npcs] / L.sum()) * 100
# Put them in desending order

# Calculate the new variance by the rotated modes
# Compute eigen values
#eof_ev = L[0:npcs]
#eof_vf = L[0:npcs] / L.sum()
#ratio = eof_vf[0] / eof_ev[0]
#vf_reofs = reofs_var * ratio
#vf_reofs = np.array([1. / float(reofs.shape[1])] * 10)
#print(vf_reofs)

#nspace = np.prod(channels)
#ev = eofs.reshape([npcs, nspace])
#print(np.shape(ev))
#field = eofs.reshape((npcs,) + originalshape)
#print(np.shape(field))
#rpcs = np.dot(ev, reofs.T)
# Should be (time(18), mode(10)) but it's (10, 10) eofs oroginally is 18 x channels (this svd computes eofs as the len of time)
# However, eofs is cut to 10 to include the 10 dominant ones beforehand so returns (10, 10)
#print(np.shape(rpcs))
#print(rpcs)

In [ ]:
# Plot the reof
tmp = reofs_des2d[0,:,:] * 100 # I can't explain the 100 scaling other than it matches
# the calculation of eof by NCL. See eof_5.ncl here https://www.ncl.ucar.edu/Applications/eof.shtml
tmp_da = xr.DataArray(tmp, coords=[jfm_sa.coords['lat'], jfm_sa.coords['lon']],
                         dims=['lat', 'lon'], name='z500')
ax = plt.axes(projection=ccrs.Orthographic(0, 90))
ax.coastlines()
ax.set_global()
tmp_da.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False)
plt.show()

This doesn't look right...

For refence I also computed the rotated eofs in NCL to see if they match. The code and plot is in the `NCL` directory and are shown here.
![title](NCL/rot_eof.png)

Enter a start-time an end-time for which to calculate the daily NAO index
e.g. lets look at the large negativate NAO of winter 2008/2009:

In [ ]:
stime = '2009-02-01-T12:00:00'
etime = '2009-02-14-T12:00:00'

Now extend for the winter months (DJF)

In [ ]:
stime = '2008-12-01-T12:00:00'
etime = '2009-02-28-T12:00:00'

Now do for all winters